# Как достать признаки с LizaAlert и оформить это в jupyter

Сначала импортируем необходимые пакеты из соответствующих библиотек

In [61]:
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as uReq
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import re
import scrapy

In [73]:
my_url = "https://lizaalert.org/forum/viewtopic.php?f=120&t=2360"
#my_url = "https://lizaalert.org/forum/viewtopic.php?f=276&t=25112"# boy and AK-47
#my_url = "https://lizaalert.org/forum/viewtopic.php?f=276&t=25087" #девочка с необычным лицом
#my_url = "https://lizaalert.org/forum/viewtopic.php?f=276&t=25197" #косоглазый чел
#my_url = "https://lizaalert.org/forum/viewtopic.php?f=276&t=25104" #ded anisimov
#my_url = "https://lizaalert.org/forum/viewtopic.php?f=276&t=25098" #ded Medvedev
#my_url = "https://lizaalert.org/forum/viewtopic.php?f=276&t=25109" #дед Павлов
#my_url = 'https://lizaalert.org/forum/viewtopic.php?f=276&t=25083'#boy Sayakbaev
#my_url = 'https://lizaalert.org/forum/viewtopic.php?f=276&t=25078' #grandma Leandrova
#my_url = "https://lizaalert.org/forum/viewtopic.php?f=276&t=25077" #girl Kamilla
client = uReq(my_url)  # создаем документ, являющийся сырым кодом сайта

In [74]:
page_html = client.read() # функция .read() позволяет вывести на экран все содержимое файла, а если мы это сохраним в переменную, 
                          # то получим объект, который хранит в себе все строки этого файла

In [75]:
client.close()

In [76]:
ps = soup(page_html, "html.parser")

In [77]:
type(ps)

bs4.BeautifulSoup

In [78]:
english_stops = ['.', ',', ':', ';', "!", "?"]

Далее, для работы понадобится открыть код страницы, которую мы хотим парсить. Для этого правой кнопки мыши нажимаем на страницу, затем выбираем просмотреть код, который затем должен появиться в правой стороне страницы.

In [79]:
containers = ps.find_all("title", {"class": "content"})
len(containers)

0

In [80]:
main = ps.find('div', {'class':'content'}).text

In [83]:
posts = ps.find('div', {'class':'pagination'})
posts
re.search(r'\d+', str(posts)).group()

<div class="pagination">
			Сообщений: 6
			 • Страница <strong>1</strong> из <strong>1</strong>
</div>

In [84]:
re.search(r'\d+', str(posts)).group()

'6'

Код для возраста 

In [81]:
title = ps.find("title").extract()
title

<title>НАЙДЕНЫ девочки 13 и 14 лет, СанктПетербург • Форум lizaalert.org</title>

In [82]:
re.search(r'\d+', str(title)).group()

'13'

In [41]:
container = containers[0]
containers[0]

<div class="content"><span style="font-size: 140%; line-height: 116%;"><span style="font-weight: bold">Казимиров Андрей Анатольевич 52 года, г. Караганда </span></span><br/><br/>9 ноября 2017 около 21:30 ушёл из дома и не вернулся<br/><br/><img alt="Изображение" src="http://info-la.ru/photo/18021_7588838613.jpg"/><br/><br/><span style="font-weight: bold">Приметы:</span><br/>Рост: 172 см.<br/>Телосложение: xудощавое<br/>Цвет глаз: карие<br/>Волосы: тёмно-русые, короткая стрижка, имеются залысины<br/><br/><span style="font-weight: bold">Был одет:</span> Клетчатая рубашка, свитер, зеленые брюки, темно-зеленая куртка, кожаная фуражка, ботинки.<br/><br/><span style="font-weight: bold">С собой:</span> Сумка через плечо<br/><br/><span style="font-weight: bold">Особые приметы:</span> Густые усы.<br/><br/>--------------------------------------------------<br/><br/><span style="color: #0000BF">Расклейка ориентировок</span></div>

In [42]:
container.img["alt"]  # проверили наличие или отсутствие изображения

'Изображение'

In [43]:
contents = container.contents
contents

[<span style="font-size: 140%; line-height: 116%;"><span style="font-weight: bold">Казимиров Андрей Анатольевич 52 года, г. Караганда </span></span>,
 <br/>,
 <br/>,
 '9 ноября 2017 около 21:30 ушёл из дома и не вернулся',
 <br/>,
 <br/>,
 <img alt="Изображение" src="http://info-la.ru/photo/18021_7588838613.jpg"/>,
 <br/>,
 <br/>,
 <span style="font-weight: bold">Приметы:</span>,
 <br/>,
 'Рост: 172 см.',
 <br/>,
 'Телосложение: xудощавое',
 <br/>,
 'Цвет глаз: карие',
 <br/>,
 'Волосы: тёмно-русые, короткая стрижка, имеются залысины',
 <br/>,
 <br/>,
 <span style="font-weight: bold">Был одет:</span>,
 ' Клетчатая рубашка, свитер, зеленые брюки, темно-зеленая куртка, кожаная фуражка, ботинки.',
 <br/>,
 <br/>,
 <span style="font-weight: bold">С собой:</span>,
 ' Сумка через плечо',
 <br/>,
 <br/>,
 <span style="font-weight: bold">Особые приметы:</span>,
 ' Густые усы.',
 <br/>,
 <br/>,
 '--------------------------------------------------',
 <br/>,
 <br/>,
 <span style="color: #0000BF">

Код, чтобы достать телесложение

In [44]:
for string in contents:
    if "елосложени" in string:
        no_stops_string = ''.join([t for t in string if t not in english_stops])
        tokens = word_tokenize(no_stops_string)
for i in range(len(tokens)):
    if "елосложени" in tokens[i]:
        if tokens[i][-1] == "я":
            body = tokens[i - 1]
        else:
            body = tokens[i + 1]
body.lower()
a = WordNetLemmatizer()
a.lemmatize(body)

'xудощавое'

In [45]:
body

'xудощавое'

Код, чтобы достать цвет волос

In [46]:

for string in contents:
    if "олосы" in string:
        no_stops_string_1 = ''.join([t for t in string if t not in english_stops])
        tokens_1 = word_tokenize(no_stops_string_1)
for i in range(len(tokens_1)):
    if "олосы" in tokens_1[i]:
        hair_colour = tokens_1[i + 1]
hair_colour.lower()
b = WordNetLemmatizer()
b.lemmatize(hair_colour)

'тёмно-русые'

In [47]:
def hair_color(contents):
    hc = ''
    for string in contents:
        if "олосы" in string:
            no_stops_string_1 = ''.join([t for t in string if t not in english_stops])
            tokens_1 = word_tokenize(no_stops_string_1)
            for i in range(len(tokens_1)):
                if "олосы" in tokens_1[i]:
                    hc = tokens_1[i + 1]
                    break
    return hc

In [48]:
hair_color(contents)

'тёмно-русые'

In [49]:
tokens_1

['Волосы', 'тёмно-русые', 'короткая', 'стрижка', 'имеются', 'залысины']

Сделаем вспомогательные словари для следующего шага

In [50]:
months_length = {"янв": 31, "фев": 28, "мар": 31, "апр": 30, "май": 31, "июн": 30, "июл": 31, "авг": 31, "сен": "30",
                "окт": 31, "ноя": 30, "дек": 31}

In [51]:
loss_find = {"время пропажи": "", "время нахождения": ""}

Код, чтобы достать время пропажи

In [52]:
for string in contents:
    if "2019" in string:
        tokeny = word_tokenize(string)
        break
lost_time = tokeny[tokeny.index("2019") - 2] + " " + tokeny[tokeny.index("2019") - 1] # здесь нужно придумать, как распространить это на все года
    

NameError: name 'tokeny' is not defined

In [ ]:
lost_time = lost_time.replace(re.search(r' \w+', lost_time).group(), re.search(r' \w{3}', lost_time).group())
lost_time

In [ ]:
loss_find["время пропажи"] = lost_time

Код, чтобы достать время нахождения

In [ ]:
postbody = ps.find('div', {"class":"notice"}).text

In [ ]:
postbody

In [ ]:
find_time = re.search(r'\d{2} \w+', str(postbody)).group()
find_time

In [ ]:
if "0" in find_time:
    find_time = find_time.replace("0", "")

In [ ]:
loss_find["время нахождения"] = find_time

In [ ]:
loss_find

# А теперь реализуем то, ради чего были найдены предыдущие два пункта: время поиска

In [ ]:
ml = list(months_length.keys())
lt = re.search(r'\w{3}', loss_find["время пропажи"]).group()
ft = re.search(r'\w{3}', loss_find["время нахождения"]).group()

In [ ]:
if lt == ft:
    search_time = int(re.search(r'\d+', loss_find["время нахождения"]).group()) - int(re,search(r'\d+', loss_find["время пропажи"]).group())
elif ml.index(ft) > ml.index(lt):
    search_time = int(re.search(r'\d+', loss_find["время нахождения"]).group())
    for i in range(ml.index(ft) - 1, ml.index(lt) - 1, -1):
        if ml[i] != lt:
            search_time += months_length[ml[i]]
        else:
            search_time += months_length[ml[i]] - int(re.search(r'\d+', loss_find["время пропажи"]).group())
elif ml.index(ft) < ml.index(lt):
    search_time = int(re.search(r'\d+', loss_find["время нахождения"]).group())
    i = ml.index(ft) - 1
    while i >= 0:
        if ml[i] != lt:
            search_time += months_length[ml[i]]
        i -= 1
    for j in range(len(ml) - 1, ml.index(lt) - 1, -1):
        if ml[j] != lt:
            search_time += months_length[ml[j]]
        else:
            search_time += months_length[ml[j]] - int(re.search(r'\d+', loss_find["время пропажи"]).group())
        
    

            
    
        
        

In [ ]:
search_time

In [ ]:
containers2 = ps.findAll("div", {"class":"content"})[0].contents
containers2

In [ ]:
containers.contents

Код для доставания возраста 